In [1]:
import sys;print(sys.version)

3.10.8 (v3.10.8:aaaf517424, Oct 11 2022, 10:14:40) [Clang 13.0.0 (clang-1300.0.29.30)]


In [1]:
os.getcwd() 

'/Users/biancaziliotto/Desktop/GitRepo/MLx2'

In [2]:
os.chdir('src')

In [3]:
from MLP import MLP 

In [4]:
mlp = MLP(0, [], 3, 2)

In [8]:
mlp.layers

In [7]:
mlp = MLP(1, [1], 3, 2)

TypeError: Layer.__init__() takes 1 positional argument but 4 were given

In [25]:
import numpy as np
X = np.zeros((4,3))
print(X)
X[0,0] = 1
print(X)
print(X[0,0])
mlp.predict(X)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
1.0


array([[0.02, 0.02],
       [0.01, 0.01],
       [0.01, 0.01],
       [0.01, 0.01]])

In [26]:
mlp.fit(X, X[:,1:], batch_size=2, error_function_str="MSE")


MSE = 0.00025
MSE = 0.0001
